# Langchain Fundamentals

In [3]:
import os
os.environ['OPENAI_API_KEY'] = 'add-your-OpenAI-key-here'

## LLMs

In [4]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
name = llm.predict("I want to open a grocery chain in Canada. Suggest a fancy name for this.")
print(name)

<ipython-input-4-27f9d123d7a4>:4: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  name = llm.predict("I want to open a grocery chain in Canada. Suggest a fancy name for this.")



"Harvest Haven"


## Prompt Templates

In [6]:
from langchain.prompts import PromptTemplate

prompt_template_country = PromptTemplate(
    input_variables =['country'],
    template = "I want to open a grocery chain in {country}. Suggest a fancy name for this."
)
p = prompt_template_country.format(country="India")
print(p)


I want to open a grocery chain in India. Suggest a fancy name for this.


## Chains

In [7]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_country)
chain.run("India")

<ipython-input-7-cb5ebac313f8>:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template_country)
<ipython-input-7-cb5ebac313f8>:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  chain.run("India")


'\n\n"Spice Emporium"'

In [24]:
chain = LLMChain(llm=llm, prompt=prompt_template_country, verbose=True)
chain.run("Canada")



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a grocery chain in Canada. Suggest a fancy name for this.

> Finished chain.


'\n\n"Maple Market Co."'

In [25]:
llm = OpenAI(temperature=0.6)

prompt_template_country = PromptTemplate(
    input_variables =['country'],
    template = "I want to open a grocery chain in {country}. Suggest a fancy name for this."
)

country_chain =LLMChain(llm=llm, prompt=prompt_template_country)

prompt_template_locations = PromptTemplate(
    input_variables = ['country'],
    template="""Suggest some prime locations in {country} where opening a grocery store could be highly profitable."""
)

locations_chain = LLMChain(llm=llm, prompt=prompt_template_locations)

## Simple Sequential Chain

In [26]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [country_chain, locations_chain])

content = chain.run("Canada")
print(content)

 

1. Downtown area: The downtown area of Maple Market Co. is usually the busiest and most populated part of the city, making it a prime location for a grocery store. This area typically has a high footfall and a diverse population, providing a steady flow of customers for the store.

2. Residential neighborhoods: Setting up a grocery store in a residential neighborhood is also a smart choice as it caters to the daily needs of the local residents. This location can also be beneficial for building a loyal customer base and creating a sense of community within the neighborhood.

3. Near schools and colleges: Placing a grocery store near schools and colleges can be a profitable venture as students often need quick and convenient options for their grocery shopping. It can also attract faculty and staff members of these institutions, providing a steady flow of customers.

4. Commercial areas: Areas with a high concentration of offices and businesses can be a great location for a grocery sto

## Sequential Chain

In [8]:
llm = OpenAI(temperature=0.7)

prompt_template_country = PromptTemplate(
    input_variables =['country'],
    template = "I want to open a grocery chain in {country}. Suggest a fancy name for this."
)

country_chain =LLMChain(llm=llm, prompt=prompt_template_country, output_key="store_name")

In [10]:
llm = OpenAI(temperature=0.7)

prompt_template_locations = PromptTemplate(
    input_variables = ['country'],
    template="""Suggest some prime locations in {country} where opening a grocery store could be highly profitable."""
)

locations_chain =LLMChain(llm=llm, prompt=prompt_template_locations, output_key="locations")

In [11]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [country_chain, locations_chain],
    input_variables = ['country'],
    output_variables = ['store_name', "locations"]
)

In [13]:
chain({"country": "Canada"})

{'country': 'Canada',
 'store_name': '\n\n"Maple Market" ',
 'locations': '\n\n1. Toronto, Ontario\n2. Vancouver, British Columbia\n3. Montreal, Quebec\n4. Calgary, Alberta\n5. Ottawa, Ontario\n6. Edmonton, Alberta\n7. Mississauga, Ontario\n8. Winnipeg, Manitoba\n9. Quebec City, Quebec\n10. Halifax, Nova Scotia'}

## Agents

In [14]:
os.environ['SERPAPI_API_KEY'] = 'add-your-serpapi-key-here'

SerpApi and LLM Math Tool

In [22]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. 
tools = load_tools(["serpapi", "llm-math"], llm=llm)

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

agent.run("What was the Population of Canada in the year 2020 and add 1 Million to the number?")



> Entering new AgentExecutor chain...
 I should use a search engine to find the population of Canada in 2020.
Action: Search
Action Input: "Population of Canada in 2020"
Observation: {'type': 'population_result', 'place': 'Canada', 'population': '38.01 million', 'year': '2020'}
Thought: I should use a calculator to add 1 million to the population.
Action: Calculator
Action Input: 38.01 million + 1 million
Observation: Answer: 39010000.0
Thought: I now know the final answer.
Final Answer: The population of Canada in 2020 was 38.01 million and adding 1 million gives a total of 39.01 million.

> Finished chain.


'The population of Canada in 2020 was 38.01 million and adding 1 million gives a total of 39.01 million.'

## Memory

In [24]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


<ipython-input-24-ffeb4aaf188e>:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  convo = ConversationChain(llm=OpenAI(temperature=0.7))


In [30]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.run("Who won the latest t20 Cricket T20 world cup?")

' The most recent T20 Cricket World Cup was won by the West Indies in 2016. They defeated England in the final match by four wickets. The tournament was held in India and it was the sixth edition of the T20 World Cup. The West Indies team was led by Darren Sammy and they had an incredible performance throughout the tournament. Before this, India won the T20 World Cup in 2014 and Sri Lanka won in 2012.'

In [31]:
convo.run("How much is 25+2?")

'  25+2 is equal to 27.'

In [32]:
convo.run("Who was the captain of the winning team?")

' I do not have enough information to answer that question. Can you provide me with the name of the team or the location of the event?'